# Collection of Indices applicable to Landsat and Sentinel imageries
### Add more details here:

In [ ]:
import os
import ee

In [ ]:
ee.Initialize()

Images need to be 'prepared' first, which consists of renaming bands to standardardized versions

In [ ]:
# 1. Calculate/addBands NDVI for an image
# @param {ee.Image} image,  e.g., Landsat or Sentinel-2 image with NIR and RED bands
# @returns {ee.Image} NDVI image
#
def calcNDVI(image):
    ndvi = (ee.Image(image).normalizedDifference(['NIR','RED'])
    .rename('NDVI'))
    return ndvi

def addNDVI(image):
    ndvi = ee.Image(image).normalizedDifference(['NIR','RED']).float()
    return image.addBands(ndvi).rename('NDVI')

In [ ]:
# 2. Calculate 'Green' NDVI for an image
# @param {ee.Image} image,  e.g., Landsat image with NIR and RED bands
# @returns {ee.Image} GNDVI image
#
def addGNDVI(image):
    gndvi = ee.Image(image).normalizedDifference(['NIR','GREEN']).float()
    return image.addBands(gndvi.rename('gNDVI'))

In [ ]:
# 3. Calculate Standard Deviation of NDVI values over a image collection
#
def addNDVIsd(image):
    ndviStd = ee.ImageCollection(image).select('NDVI').reduce(ee.Reducer.stdDev())
    # reducer = ee.Reducer.stdDev()
    # ndviStd = ndvi.reduce(ee.Reducer.median())
    # return image.addBands(ndviStd.rename('ndviSD'))
    return ndviStd.rename('ndviSD')

In [ ]:
# 4. Calculate NBR for an image
# @param {ee.Image} image  Landsat image with NIR and SWIR2 bands
# @returns {ee.Image} NBR image
#
def calcNBR(image):
    nbr = ee.Image(image).normalizedDifference(['NIR', 'SWIR2']).rename('NBR')
    return nbr

In [ ]:
# 5. Calculate EVI for an image
# @param {ee.Image} image Landsat image with NIR, RED, and BLUE bands
# @returns {ee.Image} EVI transform
#
def calcEVI(image):
    evi = ee.Image(image).expression(
          'float(2.5*(((B4) - (B3)) / ((B4) + (6 * (B3)) - (7.5 * (B1)) + 1)))',
          {
              'B4': ee.Image(image).select(['NIR']),
              'B3': ee.Image(image).select(['RED']),
              'B1': ee.Image(image).select(['BLUE'])
          }).rename('EVI')
    return evi

In [ ]:
# 6. Calculate EVI2
#
def calcEVI2(image):
    evi2 = ee.Image(image).expression(
        'float(2.5*(((B4) - (B3)) / ((B4) + (2.4 * (B3)) + 1)))',
        {
            'B4': image.select('NIR'),
            'B3': image.select('RED')
        }).rename('EVI2')
    return evi2

In [ ]:
# 7. Calculate/addBands: NDMI
#
def addNDMI(image):
    ndmi = ee.Image(image).normalizedDifference(['NIR', 'SWIR1']).float().rename('NDMI')
    return image.addBands(ndmi)

In [ ]:
# 8. Calculate/addBands: NDWI
#
def addNDWI(image):
    ndwi = ee.Image(image).normalizedDifference(['GREEN', 'NIR']).float().rename('NDWI')
    return image.addBands(ndwi)

In [ ]:
# 9. Calculate and addBands: MNDWI
#
def calcMNDWI(image):
    mndwi = ee.Image(image).normalizedDifference(['GREEN','SWIR1']).float().rename('MNDWI')
    return mndwi

def addMNDWI(image):
    mndwi = ee.Image(image).normalizedDifference(['GREEN','SWIR1']).float().rename('MNDWI')
    return image.addBands(mndwi)

In [ ]:
# 10. Calc/addBands: MVI (Mangrove Vegetation Index, see SEPAL)
#
def addMVI(image):
    mvi = ee.Image(image).expression(
        '(1.0 * (NIR - GREEN) / abs(SWIR1 - GREEN))',
        {
            'GREEN': image.select('GREEN'),
            'NIR': image.select('NIR'),
            'SWIR1': image.select('SWIR1')
        }).float().rename('MVI')
    return image.addBands(mvi)

In [ ]:
# 11. Calculate/addBands: ARI2 [Sentinel-2 only]
#
def addARI2(image):
    ari2 = ee.Image(image).expression(
        '((1.0 / GREEN) - (1.0 / Redg1)) * Redg3',
        {
            'GREEN': image.select(['GREEN']),
            'Redg1': image.select(['Redg1']),  # 'B5'
            'Redg3': image.select(['Redg3'])   # 'B7'
        }).float().rename('ARI2')
    return image.addBands(ari2.rename('ARI2'))

In [ ]:
# 12. Calculate/addBands: ChlR [Sentinel-2 only]
#
def addChlR(image):
    chlRedg = ee.Image(image).expression(
        'pow((Redg3 / Redg1), -1.0)',
        {
            'Redg1': image.select(['Redg1']),
            'Redg3': image.select(['Redg3'])
        }).float().rename('ChlR')
    return image.addBands(chlRedg)

In [ ]:
# 13.Calculate/addBands: SAVI
#
def addSAVI(image):
    savi = ee.Image(image).expression(
        '(NIR - RED)/ (NIR + RED + 0.428) * (1.0 + 0.428)',
        {
            'NIR': image.select('NIR'),
            'RED': image.select('RED')
        }).float().rename('SAVI')
    return image.addBands(savi)

In [ ]:
# 14. Calculate/addBands: BSI (bare soil index)
#
def addBSI(image):
    bsi = ee.Image(image).expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1 + RED) + (NIR + BLUE))',
        {
            'BLUE': image.select('BLUE'),
            'RED': image.select('RED'),
            'NIR': image.select('NIR'),
            'SWIR1': image.select('SWIR1')
        }).float().rename('BSI')
    return image.addBands(bsi)

In [ ]:
# 15. Calculate/addBands: IBI (index-based built-up index)
# 
def addIBI(image):
    ibiA = ee.Image(image).expression('2 * SWIR1 / (SWIR1 + NIR)',
                                      {
                                          'SWIR1': image.select('SWIR1'), # S2: 'B11'
                                          'NIR': image.select('NIR'), # S2: 'B8'
                                      }).rename('IBI_A')
    ibiB = ee.Image(image).expression('(NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1))',
                                      {
                                          'NIR': image.select('NIR'),
                                          'RED': image.select('RED'),
                                          'GREEN': image.select('GREEN'),
                                          'SWIR1': image.select('SWIR1')
                                      }).rename('IBI_B')
    ibiAB = ibiA.addBands(ibiB)
    ibi = ibiAB.normalizedDifference(['IBI_A', 'IBI_B'])
    return image.addBands(ibi.rename('IBI'))

In [ ]:
# 16. Calculate/addBands: CMRI (Combined Mangrove Recognition Index0 [(NDVI-NDWI]); Gupta et al. 2018
#
def addCMRI(image):
    ndvi = ee.Image(image).normalizedDifference(['NIR', 'RED'])
    cmri = ndvi.subtract(ee.Image(image).normalizedDifference(['GREEN', 'NIR'])).rename(['CMRI'])
    return cmri

In [ ]:
# 17. Calculate/addBands: MMRI (Modular Mangrove Recognition Index) [(MNDWI-NDVI)(MNDWI+NDVI)]; Diniz et al. 2018
#
def addMMRI(image):
    ndvi = ee.Image(image).normalizedDifference(['NIR', 'RED']).abs()
    mndwi = ee.Image(image).normalizedDifference(['GREEN','SWIR1']).abs()
    mmri = ee.Image(image).normalizedDifference(ndvi, mndwi).rename(['MMRI'])
    return mmri    

In [ ]:
# 16. Calculate/addBands: EBBI (Enhanced Built-up and Bareness Index)
#
def addEBBI(image):
    ebbi = ee.Image(image).expression('(SWIR1 - NIR) / (10 * sqrt(SWIR1 + SWIR2))',
                                       {
                                           'NIR': image.select('NIR'),
                                           'SWIR1': image.select('SWIR1'),
                                           'SWIR2': image.select('SWIR2')
                                       }).rename(['EBBI'])
    return image.addBands(ebbi)

In [ ]:
 # 16. Calculate NDFI using endmembers from Souza et al., 2005
 # @param {ee.Image} Surface reflectance image with 6 bands (i.e. not thermal)
 # @returns {ee.Image} NDFI transform
 #
def calcNDFI(image):
  # Do spectral unmixing #
    gv = [.0500, .0900, .0400, .6100, .3000, .1000]
    shade = [0, 0, 0, 0, 0, 0]
    npv = [.1400, .1700, .2200, .3000, .5500, .3000]
    soil = [.2000, .3000, .3400, .5800, .6000, .5800]
    cloud = [.9000, .9600, .8000, .7800, .7200, .6500]
    cf = .1 # Not parameterized
    cfThreshold = ee.Image.constant(cf)
    unmixImage = (ee.Image(image).unmix([gv, shade, npv, soil, cloud], True,True)
                  .rename(['band_0', 'band_1', 'band_2','band_3','band_4']))
    newImage = ee.Image(image).addBands(unmixImage)
    mask = unmixImage.select('band_4').lt(cfThreshold) # Check that this is the right 'image'; previously 'Image'
    ndfi = ee.Image(unmixImage).expression(
        '((GV / (1 - SHADE)) - (NPV + SOIL)) / ((GV / (1 - SHADE)) + NPV + SOIL)', 
        {
            'GV': ee.Image(unmixImage).select('band_0'),
            'SHADE': ee.Image(unmixImage).select('band_1'),
            'NPV': ee.Image(unmixImage).select('band_2'),
            'SOIL': ee.Image(unmixImage).select('band_3')
    })
    return ee.Image(newImage) \
        .addBands(ee.Image(ndfi).rename(['NDFI'])) \
        .select(['band_0','band_1','band_2','band_3','NDFI']) \
        .rename(['GV','Shade','NPV','Soil','NDFI']) \
        .updateMask(mask)

In [ ]:
# 17. Tassel Cap coefficients from Crist 1985
# @param {ee.Image} image, Landsat image with BLUE, GREEN, RED, NIR, SWIR1, and SWIR2
# @returns {ee.Image} 3-band image with Brightness, Greenness, and Wetness
#
def tcTrans(image):
    # Calculate tasseled cap transformation
    brightness = image.expression(
        '(L1 * B1) + (L2 * B2) + (L3 * B3) + (L4 * B4) + (L5 * B5) + (L6 * B6)',
        {
            'L1': image.select('BLUE'),
            'B1': 0.2043,
            'L2': image.select('GREEN'),
            'B2': 0.4158,
            'L3': image.select('RED'),
            'B3': 0.5524,
            'L4': image.select('NIR'),
            'B4': 0.5741,
            'L5': image.select('SWIR1'),
            'B5': 0.3124,
            'L6': image.select('SWIR2'),
            'B6': 0.2303
        })
    greenness = image.expression(
        '(L1 * B1) + (L2 * B2) + (L3 * B3) + (L4 * B4) + (L5 * B5) + (L6 * B6)',
        {
            'L1': image.select('BLUE'),
            'B1': -0.1603,
            'L2': image.select('GREEN'),
            'B2': -0.2819,
            'L3': image.select('RED'),
            'B3': -0.4934,
            'L4': image.select('NIR'),
            'B4': 0.7940,
            'L5': image.select('SWIR1'),
            'B5': -0.0002,
            'L6': image.select('SWIR2'),
            'B6': -0.1446
        })
    wetness = image.expression(
        '(L1 * B1) + (L2 * B2) + (L3 * B3) + (L4 * B4) + (L5 * B5) + (L6 * B6)',
        {
            'L1': image.select('BLUE'),
            'B1': 0.0315,
            'L2': image.select('GREEN'),
            'B2': 0.2021,
            'L3': image.select('RED'),
            'B3': 0.3102,
            'L4': image.select('NIR'),
            'B4': 0.1594,
            'L5': image.select('SWIR1'),
            'B5': -0.6806,
            'L6': image.select('SWIR2'),
            'B6': -0.6109
        })

    bright =  ee.Image(brightness).rename('BRIGHTNESS')
    green = ee.Image(greenness).rename('GREENNESS')
    wet = ee.Image(wetness).rename('WETNESS')

    tasseledCap = ee.Image([bright, green, wet])
    return tasseledCap